In [1]:
from STvAR import *
import STvARFileWrite as stfw

In [2]:
DiffOrder = 4
nghostcells = 4
KOsigma = 0.3
DiffOrderUpDn = 4

In [3]:
dim = 3
path = "../Source/Z4c/"

# Generating RHS

In [4]:
fileRHS = open(path+"ET_Integration_Rhs_K.H", "w+")

In [5]:
fileRHS.write(stfw.RHS_Header())

743

In [6]:
stVar.declState = []  #We want a list of declared variables that we will cycle through when we convert our RHS equations.

chi = stVar('chi', state = True) #Scalar variables like chi can be declared with stVar.
gamtildeLL = stVarRank2('gamtildeLL', sym = 'sym01', state = True)  #gamtildeLL is rank 2, so it is declared with stVarRank2
Khat = stVar('Khat', state = True)
AtildeLL = stVarRank2('AtildeLL', sym = 'sym01', state = True)
theta = stVar('theta', state = True)
GamtildeU = stVarRank1('GamtildeU', state = True)

alpha = stVar('alpha', state = True)
betaU = stVarRank1('betaU', state = True)

In [7]:
fileVARS = open(path+"ET_Integration_Variables.H","w+")
fileVARS.write(stfw.VarString(stVar.declState))
fileVARS.close()

In [8]:
fileRHS.write(chi.AMReXSymb2State())
fileRHS.write(gamtildeLL.AMReXSymb2State())
fileRHS.write(Khat.AMReXSymb2State())
fileRHS.write(AtildeLL.AMReXSymb2State())
fileRHS.write(theta.AMReXSymb2State())
fileRHS.write(GamtildeU.AMReXSymb2State())

fileRHS.write(alpha.AMReXSymb2State())
fileRHS.write(betaU.AMReXSymb2State())

186

In [9]:
betaUp = stVarRank1('betaUp')
betaDn = stVarRank1('betaDn')

In [10]:
betaString = """
        std::vector<amrex::Real> UpWind; UpWind.push_back(1); UpWind.push_back(0); 
        std::vector<amrex::Real> DnWind; DnWind.push_back(0); DnWind.push_back(1);

        std::vector<amrex::Real> betaAdv0;
        betaAdv0.resize(2);
        
        std::vector<amrex::Real> betaAdv1;
        betaAdv1.resize(2);
        
        std::vector<amrex::Real> betaAdv2;
        betaAdv2.resize(2);
        
        betaAdv0 = (betaU0 > 0) ? UpWind : DnWind;
        betaAdv1 = (betaU1 > 0) ? UpWind : DnWind;
        betaAdv2 = (betaU2 > 0) ? UpWind : DnWind;
        
        amrex::Real betaUp0 = betaAdv0[0];
        amrex::Real betaDn0 = betaAdv0[1];
        amrex::Real betaUp1 = betaAdv1[0];
        amrex::Real betaDn1 = betaAdv1[1];
        amrex::Real betaUp2 = betaAdv2[0];
        amrex::Real betaDn2 = betaAdv2[1];

"""
fileRHS.write(betaString)

830

In [11]:
if DiffOrderUpDn%2 == 0:
    strUpCn = 'up'
else:
    strUpCn = 'cn'

betaDchi = stVar('betaDchi')
Dupchi = stVarRank1('Dupchi')
Dupchi.expr = DstVar(chi, difftype = 1,CnUpDnRank1=strUpCn,orderD = DiffOrderUpDn).expr

Ddnchi = stVarRank1('Ddnphi')
Ddnchi.expr = DstVar(chi, difftype = 1,CnUpDnRank1='dn',orderD = DiffOrderUpDn).expr

for k in range(3):
    betaDchi.expr += betaU.symb[k]*(betaUp.symb[k]*Dupchi.symb[k]+betaDn.symb[k]*Ddnchi.symb[k])
    
betaDgamtildeLL = stVarRank2('betaDgamtildeLL')
DupgamtildeLL = stVarRank3('DupgamtildeLL')
DupgamtildeLL.expr = DstVarRank2(gamtildeLL,difftype = 1, CnUpDnRank1=strUpCn,orderD=DiffOrderUpDn).expr

DdngamtildeLL = stVarRank3('DdngamtildeLL')
DdngamtildeLL.expr = DstVarRank2(gamtildeLL,difftype = 1, CnUpDnRank1='dn',orderD=DiffOrderUpDn).expr

for i in range(3):
    for j in range(3):
        for k in range(3):
            betaDgamtildeLL.expr[i][j] += betaU.symb[k]*(betaUp.symb[k]*DupgamtildeLL.symb[i][j][k]+betaDn.symb[k]*DdngamtildeLL.symb[i][j][k])

betaDKhat = stVar('betaDKhat')
DupKhat = stVarRank1('DupKhat')
DupKhat.expr = DstVar(Khat, difftype = 1,CnUpDnRank1=strUpCn,orderD = DiffOrderUpDn).expr

DdnKhat = stVarRank1('DdnKhat')
DdnKhat.expr = DstVar(Khat, difftype = 1,CnUpDnRank1='dn',orderD = DiffOrderUpDn).expr

for k in range(3):
    betaDKhat.expr += betaU.symb[k]*(betaUp.symb[k]*DupKhat.symb[k]+betaDn.symb[k]*DdnKhat.symb[k])
    
betaDAtildeLL = stVarRank2('betaDAtildeLL')
DupAtildeLL = stVarRank3('DupAtildeLL')
DupAtildeLL.expr = DstVarRank2(AtildeLL,difftype = 1, CnUpDnRank1=strUpCn,orderD=DiffOrderUpDn).expr

DdnAtildeLL = stVarRank3('DdnAtildeLL')
DdnAtildeLL.expr = DstVarRank2(AtildeLL,difftype = 1, CnUpDnRank1='dn',orderD=DiffOrderUpDn).expr

for i in range(3):
    for j in range(3):
        for k in range(3):
            betaDAtildeLL.expr[i][j] += betaU.symb[k]*(betaUp.symb[k]*DupAtildeLL.symb[i][j][k]+betaDn.symb[k]*DdnAtildeLL.symb[i][j][k])

betaDtheta = stVar('betaDtheta')
Duptheta = stVarRank1('Duptheta')
Duptheta.expr = DstVar(theta, difftype = 1,CnUpDnRank1=strUpCn,orderD = DiffOrderUpDn).expr

Ddntheta = stVarRank1('Ddntheta')
Ddntheta.expr = DstVar(theta, difftype = 1,CnUpDnRank1='dn',orderD = DiffOrderUpDn).expr

for k in range(3):
    betaDtheta.expr += betaU.symb[k]*(betaUp.symb[k]*Duptheta.symb[k]+betaDn.symb[k]*Ddntheta.symb[k])
    
betaDGamtildeU = stVarRank1('betaDGamtildeU')
DupGamtildeU = stVarRank2('DupGamtildeU')
DupGamtildeU.expr = DstVarRank1(GamtildeU,difftype = 1, CnUpDnRank1=strUpCn,orderD=DiffOrderUpDn).expr

DdnGamtildeU = stVarRank2('DdnGamtildeU')
DdnGamtildeU.expr = DstVarRank1(GamtildeU,difftype = 1, CnUpDnRank1='dn',orderD=DiffOrderUpDn).expr

for i in range(3):
    for k in range(3):
        betaDGamtildeU.expr[i] += betaU.symb[k]*(betaUp.symb[k]*DupGamtildeU.symb[i][k]+betaDn.symb[k]*DdnGamtildeU.symb[i][k])

betaDalpha = stVar('betaDalpha')
Dupalpha = stVarRank1('Dupalpha')
Dupalpha.expr = DstVar(alpha, difftype = 1,CnUpDnRank1=strUpCn,orderD = DiffOrderUpDn).expr

Ddnalpha = stVarRank1('Ddnalpha')
Ddnalpha.expr = DstVar(alpha, difftype = 1,CnUpDnRank1='dn',orderD = DiffOrderUpDn).expr

for k in range(3):
    betaDalpha.expr += betaU.symb[k]*(betaUp.symb[k]*Dupalpha.symb[k]+betaDn.symb[k]*Ddnalpha.symb[k])
    
betaDbetaU = stVarRank1('betaDbetaU')
DupbetaU = stVarRank2('DupbetaU')
DupbetaU.expr = DstVarRank1(betaU,difftype = 1, CnUpDnRank1=strUpCn,orderD=DiffOrderUpDn).expr

DdnbetaU = stVarRank2('DdnbetaU')
DdnbetaU.expr = DstVarRank1(betaU,difftype = 1, CnUpDnRank1='dn',orderD=DiffOrderUpDn).expr

for i in range(3):
    for k in range(3):
        betaDbetaU.expr[i] += betaU.symb[k]*(betaUp.symb[k]*DupbetaU.symb[i][k]+betaDn.symb[k]*DdnbetaU.symb[i][k])


In [12]:
fileRHS.write(Dupchi.AMReXSymb2Expr())
fileRHS.write(Ddnchi.AMReXSymb2Expr())
fileRHS.write(betaDchi.AMReXSymb2Expr())

fileRHS.write(DupgamtildeLL.AMReXSymb2Expr())
fileRHS.write(DdngamtildeLL.AMReXSymb2Expr())
fileRHS.write(betaDgamtildeLL.AMReXSymb2Expr())

fileRHS.write(DupKhat.AMReXSymb2Expr())
fileRHS.write(DdnKhat.AMReXSymb2Expr())
fileRHS.write(betaDKhat.AMReXSymb2Expr())

fileRHS.write(DupAtildeLL.AMReXSymb2Expr())
fileRHS.write(DdnAtildeLL.AMReXSymb2Expr())
fileRHS.write(betaDAtildeLL.AMReXSymb2Expr())

fileRHS.write(Duptheta.AMReXSymb2Expr())
fileRHS.write(Ddntheta.AMReXSymb2Expr())
fileRHS.write(betaDtheta.AMReXSymb2Expr())

fileRHS.write(DupGamtildeU.AMReXSymb2Expr())
fileRHS.write(DdnGamtildeU.AMReXSymb2Expr())
fileRHS.write(betaDGamtildeU.AMReXSymb2Expr())

fileRHS.write(Dupalpha.AMReXSymb2Expr())
fileRHS.write(Ddnalpha.AMReXSymb2Expr())
fileRHS.write(betaDalpha.AMReXSymb2Expr())

fileRHS.write(DupbetaU.AMReXSymb2Expr())
fileRHS.write(DdnbetaU.AMReXSymb2Expr())
fileRHS.write(betaDbetaU.AMReXSymb2Expr())

558

In [13]:
fileRHS.write(DstVar(chi, 1, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVar(chi, 2, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVar(chi, 'KO', orderD = DiffOrder, sigma = KOsigma).AMReXSymb2Expr())

fileRHS.write(DstVarRank2(gamtildeLL,1,orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVarRank2(gamtildeLL,2,orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVarRank2(gamtildeLL, 'KO', orderD = DiffOrder, sigma = KOsigma).AMReXSymb2Expr())

fileRHS.write(DstVar(Khat,1,orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVar(Khat, 'KO', sigma = KOsigma,orderD = DiffOrder).AMReXSymb2Expr())

fileRHS.write(DstVarRank2(AtildeLL,1, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVarRank2(AtildeLL, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXSymb2Expr())

fileRHS.write(DstVar(theta,1, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVar(theta, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXSymb2Expr())

fileRHS.write(DstVarRank1(GamtildeU, 1, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVarRank1(GamtildeU, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXSymb2Expr())

fileRHS.write(DstVar(alpha, 1, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVar(alpha, 2, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVar(alpha, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXSymb2Expr())

fileRHS.write(DstVarRank1(betaU, 1, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVarRank1(betaU, 2, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVarRank1(betaU, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXSymb2Expr())

2835

In [14]:
gamtildeUU = stVarRank2('gamtildeUU')
for i in range(dim):
    for j in range(dim):
         gamtildeUU.expr[i][j] += sp.simplify(sp.Matrix(gamtildeLL.symb).inv()[i,j])

In [15]:
fileRHS.write(gamtildeUU.AMReXSymb2Expr())

3026

In [16]:
DgamtildeLLL = stVarRank3('DgambarLLL')
GamtildeULL = stVarRank3('GamtildeULL')
GamtildeLLL = stVarRank3('GamtildeLLL')
GamULL = stVarRank3('GamULL')

for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            DgamtildeLLL.expr[i][j][k] = Dsymb(gamtildeLL.symb[i][j],k)
            
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            for l in range(dim):
                GamtildeULL.expr[i][j][k] += 1/2*gamtildeUU.symb[i][l]*(DgamtildeLLL.symb[l][j][k]+DgamtildeLLL.symb[l][k][j]-DgamtildeLLL.symb[j][k][l])
                
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            for l in range(dim):
                GamtildeLLL.expr[i][j][k] += gamtildeLL.symb[i][l]*GamtildeULL.symb[l][j][k]
                
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            GamULL.expr[i][j][k] += GamtildeULL.symb[i][j][k] - 1/2*((sp.eye(3)[i,j]/chi.symb)*Dsymb(chi.symb,k)+(sp.eye(3)[i,k]/chi.symb)*Dsymb(chi.symb,j))
            for l in range(3):
                GamULL.expr[i][j][k] += 1/2*gamtildeLL.symb[j][k]*gamtildeUU.symb[i][l]*Dsymb(chi.symb,l)/chi.symb


In [17]:
fileRHS.write(DgamtildeLLL.AMReXSymb2Expr())
fileRHS.write(GamtildeULL.AMReXSymb2Expr())
fileRHS.write(GamtildeLLL.AMReXSymb2Expr())
fileRHS.write(GamULL.AMReXSymb2Expr())

4938

In [18]:
DDalphaLL = stVarRank2('DDalphaLL')
for i in range(3):
    for j in range(3):
        DDalphaLL.expr[i][j] += Ds(alpha,str(i)+str(j))
        for k in range(3):
            DDalphaLL.expr[i][j] += -GamULL.symb[k][i][j]*Ds(alpha,str(k))

In [19]:
fileRHS.write(DDalphaLL.AMReXSymb2Expr())

954

In [20]:
gamLL = stVarRank2('gamLL')
for i in range(3):
    for j in range(3):
        gamLL.expr[i][j] = (chi.symb**(-1))*gamtildeLL.symb[i][j]
        
gamUU = stVarRank2('gamUU')
for i in range(3):
    for j in range(3):
        gamUU.expr[i][j] = chi.symb*gamtildeUU.symb[i][j]

In [21]:
fileRHS.write(gamLL.AMReXSymb2Expr())
fileRHS.write(gamUU.AMReXSymb2Expr())

432

In [22]:
AtildeUU = stVarRank2('AtildeUU')
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                AtildeUU.expr[i][j] += gamtildeUU.symb[i][k]*gamtildeUU.symb[j][l]*AtildeLL.symb[k][l]
                
AtildeUL = stVarRank2('AtildeUL')
for i in range(3):
    for j in range(3):
        for k in range(3):
            AtildeUL.expr[i][j] += gamtildeUU.symb[i][k]*AtildeLL.symb[k][j]

In [23]:
fileRHS.write(AtildeUU.AMReXSymb2Expr())
fileRHS.write(AtildeUL.AMReXSymb2Expr())

990

In [24]:
DDalphaTFLL=stVarRank2('DDalphaTFLL')

for i in range(3):
    for j in range(3):
        DDalphaTFLL.expr[i][j] += DDalphaLL.symb[i][j]
        for k in range(3):
            for l in range(3):
                DDalphaTFLL.expr[i][j] += -1/3*gamLL.symb[i][j]*(gamUU.symb[k][l]*DDalphaLL.symb[k][l])

In [25]:
fileRHS.write(DDalphaTFLL.AMReXSymb2Expr())

4500

In [26]:
GamtildeDU = stVarRank1('GamtildeDU')
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            GamtildeDU.expr[i] += GamtildeULL.symb[i][j][k]*gamtildeUU.symb[j][k] 

In [27]:
fileRHS.write(GamtildeDU.AMReXSymb2Expr())

909

In [28]:
RtildeLL = stVarRank2('RtildeLL')

for i in range(3):
    for j in range(3):
        for l in range(3):
            for m in range(3):
                RtildeLL.expr[i][j] += -1/2*gamtildeUU.symb[l][m]*Dsymb(gamtildeLL.symb[i][j],str(l)+str(m))
        for k in range(3):
            RtildeLL.expr[i][j] += 1/2*(gamtildeLL.symb[k][i]*Dsymb(GamtildeU.symb[k],str(j))+gamtildeLL.symb[k][j]*Dsymb(GamtildeU.symb[k],str(i)))
            RtildeLL.expr[i][j] += 1/2*(GamtildeDU.symb[k]*GamtildeLLL.symb[i][j][k]+GamtildeDU.symb[k]*GamtildeLLL.symb[j][i][k])
            
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                for m in range(3):
                    RtildeLL.expr[i][j] += gamtildeUU.symb[l][m]*(GamtildeULL.symb[k][l][i]*GamtildeLLL.symb[j][k][m]+GamtildeULL.symb[k][l][j]*GamtildeLLL.symb[i][k][m])
                    RtildeLL.expr[i][j] += gamtildeUU.symb[l][m]*GamtildeULL.symb[k][i][m]*GamtildeLLL.symb[k][l][j]

In [29]:
fileRHS.write(RtildeLL.AMReXSymb2Expr())

33534

In [30]:
DtildeDtildechiLL = stVarRank2('DtildeDtildechiLL')
for i in range(3):
    for j in range(3):
        DtildeDtildechiLL.expr[i][j] += Dsymb(chi.symb,str(i)+str(j))
        for k in range(3):
            DtildeDtildechiLL.expr[i][j] += -GamtildeULL.symb[k][i][j]*Dsymb(chi.symb,k)
            
TrDtildeDtildechi = stVar('TrDtildeDtildechi')
for i in range(3):
    for j in range(3):
        TrDtildeDtildechi.expr += gamtildeUU.symb[i][j]*DtildeDtildechiLL.symb[i][j]

In [31]:
fileRHS.write(DtildeDtildechiLL.AMReXSymb2Expr())
fileRHS.write(TrDtildeDtildechi.AMReXSymb2Expr())

355

In [32]:
RchiLL = stVarRank2('RchiLL')            
for i in range(3):
    for j in range(3):
        RchiLL.expr[i][j] += 1/2*(chi.symb**(-1))*(DtildeDtildechiLL.symb[i][j]+gamtildeLL.symb[i][j]*TrDtildeDtildechi.symb)
        RchiLL.expr[i][j] += -1/4*(chi.symb**(-2))*Dsymb(chi.symb,str(i))*Dsymb(chi.symb,str(j))
        for l in range(3):
            for m in range(3):
                RchiLL.expr[i][j] += -3/4*(chi.symb**(-2))*gamtildeLL.symb[i][j]*gamtildeUU.symb[l][m]*Dsymb(chi.symb,str(m))*Dsymb(chi.symb,str(l))
        

In [33]:
fileRHS.write(RchiLL.AMReXSymb2Expr())

6411

In [34]:
RLL = stVarRank2('RLL')
for i in range(3):
    for j in range(3):
        RLL.expr[i][j] += RtildeLL.symb[i][j]+RchiLL.symb[i][j]

In [35]:
fileRHS.write(RLL.AMReXSymb2Expr())

459

In [36]:
Rsclr = stVar('Rsclr')
for i in range(3):
    for j in range(3):
        Rsclr.expr += gamUU.symb[i][j]*RLL.symb[i][j] 

In [37]:
fileRHS.write(Rsclr.AMReXSymb2Expr())

172

In [38]:
RTFLL = stVarRank2('RTFLL')
for i in range(3):
    for j in range(3):
        RTFLL.expr[i][j] = RLL.symb[i][j]-1/3*gamLL.symb[i][j]*Rsclr.symb

In [39]:
fileRHS.write(RTFLL.AMReXSymb2Expr())

657

In [40]:
divTildebeta = stVar('divTildebeta')
for i in range(3):
    divTildebeta.expr += Dsymb(betaU.symb[i],str(i))
    for k in range(3):
        divTildebeta.expr += GamtildeULL.symb[i][i][k]*betaU.symb[k]

In [41]:
fileRHS.write(divTildebeta.AMReXSymb2Expr())

284

In [42]:
RHS_chi = stVar('chi')

RHS_chi.expr = 2.0/3*chi.symb*(alpha.symb*(Khat.symb+2*theta.symb)-divTildebeta.symb)
RHS_chi.expr += betaDchi.symb     
RHS_chi.expr += Dsymb(chi.symb,'KO')

In [43]:
fileRHS.write(RHS_chi.AMReXSetRHS())

122

In [44]:
RHS_gamtildeLL = stVarRank2('gamtildeLL',sym = 'sym01')

for i in range(3):
    for j in range(3):
        RHS_gamtildeLL.expr[i][j] += -2*alpha.symb*AtildeLL.symb[i][j]
        RHS_gamtildeLL.expr[i][j] += betaDgamtildeLL.symb[i][j]
        for k in range(3):
            RHS_gamtildeLL.expr[i][j] += gamtildeLL.symb[k][i]*Dsymb(betaU.symb[k],str(j))+gamtildeLL.symb[k][j]*Dsymb(betaU.symb[k],str(i))
            RHS_gamtildeLL.expr[i][j] += -2/3*gamtildeLL.symb[i][j]*Dsymb(betaU.symb[k],str(k))
            
for i in range(3):
    for j in range(3):
        RHS_gamtildeLL.expr[i][j] += Dsymb(gamtildeLL.symb[i][j],'KO')

In [45]:
fileRHS.write(RHS_gamtildeLL.AMReXSetRHS())

1935

In [46]:
kappa1 = stVar('kappa1')
kappa2 = stVar('kappa2')

kappa1.expr = 0.02
kappa2.expr = 0

RHS_Khat = stVar('Khat')
for i in range(3):
    for j in range(3):
        RHS_Khat.expr += -gamUU.symb[i][j]*DDalphaLL.symb[j][i]+alpha.symb*AtildeLL.symb[i][j]*AtildeUU.symb[i][j]

RHS_Khat.expr += betaDKhat.symb
RHS_Khat.expr += 1/3*alpha.symb*((Khat.symb+2*theta.symb)**2+3*kappa1.symb*(1-kappa2.symb)*theta.symb)
RHS_Khat.expr += Dsymb(Khat.symb,'KO')

In [47]:
fileRHS.write(kappa1.AMReXSymb2Expr())
fileRHS.write(kappa2.AMReXSymb2Expr())
fileRHS.write(RHS_Khat.AMReXSetRHS())

615

In [48]:
RHS_AtildeLL = stVarRank2('AtildeLL', sym = 'sym01')

for i in range(3):
    for j in range(3):
        RHS_AtildeLL.expr[i][j] += chi.symb*(-DDalphaTFLL.symb[i][j]+alpha.symb*RTFLL.symb[i][j])+alpha.symb*(Khat.symb+2*theta.symb)*AtildeLL.symb[i][j]
        RHS_AtildeLL.expr[i][j] += betaDAtildeLL.symb[i][j]
        for k in range(3):
            RHS_AtildeLL.expr[i][j] += -2*alpha.symb*AtildeLL.symb[i][k]*AtildeUL.symb[k][j]
            RHS_AtildeLL.expr[i][j] += AtildeLL.symb[k][i]*Dsymb(betaU.symb[k],str(j))+AtildeLL.symb[k][j]*Dsymb(betaU.symb[k],str(i))-2/3*AtildeLL.symb[i][j]*Dsymb(betaU.symb[k],str(k))
            
for i in range(3):
    for j in range(3):
        RHS_AtildeLL.expr[i][j] += Dsymb(AtildeLL.symb[i][j],'KO')

In [49]:
fileRHS.write(RHS_AtildeLL.AMReXSetRHS())

2718

In [50]:
RHS_theta = stVar('theta')

RHS_theta.expr = 1/2*alpha.symb*(Rsclr.symb + 2/3*(Khat.symb+2*theta.symb)**2 - 2*kappa1.symb*(2+kappa2.symb)*theta.symb)
RHS_theta.expr += betaDtheta.symb
for i in range(3):
    for j in range(3):
        RHS_theta.expr += -1/2*alpha.symb*AtildeLL.symb[i][j]*AtildeUU.symb[i][j]

RHS_theta.expr += Dsymb(theta.symb,'KO')

In [51]:
fileRHS.write(RHS_theta.AMReXSetRHS())

469

In [52]:
RHS_GamtildeU = stVarRank1('GamtildeU')

for i in range(3):
    RHS_GamtildeU.expr[i] += betaDGamtildeU.symb[i]
    for j in range(3):
        RHS_GamtildeU.expr[i] += -2*AtildeUU.symb[i][j]*Dsymb(alpha.symb,str(j))+2*alpha.symb*(-1/3*gamtildeUU.symb[i][j]*(2*Dsymb(Khat.symb,str(j))+Dsymb(theta.symb,str(j)))-3/2*AtildeUU.symb[i][j]*Dsymb(chi.symb,str(j))/chi.symb)
        RHS_GamtildeU.expr[i] += -GamtildeDU.symb[j]*Dsymb(betaU.symb[i],str(j))+2/3*GamtildeDU.symb[i]*Dsymb(betaU.symb[j],str(j))
        for k in range(3):
            RHS_GamtildeU.expr[i] += 2*alpha.symb*GamtildeULL.symb[i][j][k]*AtildeUU.symb[j][k]
            RHS_GamtildeU.expr[i] += 1/3*gamtildeUU.symb[i][j]*Dsymb(betaU.symb[k],str(j)+str(k))+gamtildeUU.symb[j][k]*Dsymb(betaU.symb[i],str(j)+str(k))
            
for i in range(3):
    RHS_GamtildeU.expr[i] += -2*alpha.symb*kappa1.symb*(GamtildeU.symb[i]-GamtildeDU.symb[i])
for i in range(3):
    RHS_GamtildeU.expr[i] += Dsymb(GamtildeU.symb[i],'KO')

In [163]:
print(RHS_GamtildeU.AMReXSetRHS())

        rhs_fab(i, j, k, Idx::GamtildeU0) = 2*AtildeUU00*GamtildeULL000*alpha - 2*AtildeUU00*dalpha0 + 2*AtildeUU01*GamtildeULL001*alpha - 2*AtildeUU01*dalpha1 + 2*AtildeUU02*GamtildeULL002*alpha - 2*AtildeUU02*dalpha2 + 2*AtildeUU10*GamtildeULL010*alpha + 2*AtildeUU11*GamtildeULL011*alpha + 2*AtildeUU12*GamtildeULL012*alpha + 2*AtildeUU20*GamtildeULL020*alpha + 2*AtildeUU21*GamtildeULL021*alpha + 2*AtildeUU22*GamtildeULL022*alpha - 0.33333333333333337*GamtildeDU0*dbetaU00 + 0.66666666666666663*GamtildeDU0*dbetaU11 + 0.66666666666666663*GamtildeDU0*dbetaU22 - GamtildeDU1*dbetaU01 - GamtildeDU2*dbetaU02 + KOGamtildeU0 - 2*alpha*kappa1*(-GamtildeDU0 + GamtildeU0) + 2*alpha*(-1.5*AtildeUU00*dchi0/chi - 0.33333333333333331*gamtildeUU00*(2*dKhat0 + dtheta0)) + 2*alpha*(-1.5*AtildeUU01*dchi1/chi - 0.33333333333333331*gamtildeUU01*(2*dKhat1 + dtheta1)) + 2*alpha*(-1.5*AtildeUU02*dchi2/chi - 0.33333333333333331*gamtildeUU02*(2*dKhat2 + dtheta2)) + betaDGamtildeU0 + 1.3333333333333333*ddbetaU00

In [53]:
fileRHS.write(RHS_GamtildeU.AMReXSetRHS())

4779

In [54]:
mul = stVar('mul')
mus = stVar('mus')

mul.expr = 2/alpha.symb
mus.expr = 1/(alpha.symb**(2))

In [55]:
RHS_alpha = stVar('alpha')

RHS_alpha.expr += -mul.symb*(alpha.symb**2)*Khat.symb+betaDalpha.symb
RHS_alpha.expr += Dsymb(alpha.symb,'KO')

In [56]:
fileRHS.write(mul.AMReXSymb2Expr())
fileRHS.write(mus.AMReXSymb2Expr())
fileRHS.write(RHS_alpha.AMReXSetRHS())

92

In [57]:
eta = stVar('eta')
eta.expr = 2.0

RHS_betaU = stVarRank1('betaU')

for i in range(3):
    RHS_betaU.expr[i] += mus.symb*(alpha.symb**2)*GamtildeU.symb[i]-eta.symb*betaU.symb[i]+betaDbetaU.symb[i]
    
for i in range(3):
    RHS_betaU.expr[i] += Dsymb(betaU.symb[i],'KO')

In [58]:
fileRHS.write(eta.AMReXSymb2Expr())
fileRHS.write(RHS_betaU.AMReXSetRHS())

342

In [59]:
fileRHS.write(stfw.Closer())
fileRHS.close()

# Diagnostics file

In [60]:
fileDIAG = open(path+"ET_Integration_Diag_K.H", "w+")

In [61]:
fileDIAG.write(stfw.Diag_Header())

754

In [62]:
stVar.declDiag=[]
H = stVar('H', diag=True)
ML = stVarRank1('ML', diag=True)
C = stVar('C', diag=True)
Ksclr = stVar('Ksclr', diag=True)

In [63]:
fileDIAG.write(chi.AMReXSymb2State())
fileDIAG.write(gamtildeLL.AMReXSymb2State())
fileDIAG.write(Khat.AMReXSymb2State())
fileDIAG.write(AtildeLL.AMReXSymb2State())
fileDIAG.write(theta.AMReXSymb2State())
fileDIAG.write(GamtildeU.AMReXSymb2State())

210

In [64]:
fileDIAG.write(DstVar(chi, 1, orderD = DiffOrder).AMReXSymb2Expr())
fileDIAG.write(DstVar(chi, 2, orderD = DiffOrder).AMReXSymb2Expr())

fileDIAG.write(DstVarRank2(gamtildeLL,1,orderD = DiffOrder).AMReXSymb2Expr())
fileDIAG.write(DstVarRank2(gamtildeLL,2,orderD = DiffOrder).AMReXSymb2Expr())

fileDIAG.write(DstVar(Khat,1,orderD = DiffOrder).AMReXSymb2Expr())

fileDIAG.write(DstVarRank2(AtildeLL,1, orderD = DiffOrder).AMReXSymb2Expr())

fileDIAG.write(DstVar(theta,1,orderD = DiffOrder).AMReXSymb2Expr())

fileDIAG.write(DstVarRank1(GamtildeU, 1, orderD = DiffOrder).AMReXSymb2Expr())

2232

In [65]:
fileDIAG.write(gamtildeUU.AMReXSymb2Expr())

3026

In [66]:
fileDIAG.write(DgamtildeLLL.AMReXSymb2Expr())
fileDIAG.write(GamtildeULL.AMReXSymb2Expr())
fileDIAG.write(GamtildeLLL.AMReXSymb2Expr())
fileDIAG.write(GamULL.AMReXSymb2Expr())

4938

In [67]:
fileDIAG.write(gamLL.AMReXSymb2Expr())
fileDIAG.write(gamUU.AMReXSymb2Expr())

432

In [68]:
fileDIAG.write(AtildeUU.AMReXSymb2Expr())
fileDIAG.write(AtildeUL.AMReXSymb2Expr())

990

In [69]:
fileDIAG.write(GamtildeDU.AMReXSymb2Expr())

909

In [70]:
fileDIAG.write(RtildeLL.AMReXSymb2Expr())

33534

In [71]:
fileDIAG.write(DtildeDtildechiLL.AMReXSymb2Expr())
fileDIAG.write(TrDtildeDtildechi.AMReXSymb2Expr())

355

In [72]:
fileDIAG.write(RchiLL.AMReXSymb2Expr())

6411

In [73]:
fileDIAG.write(RLL.AMReXSymb2Expr())
fileDIAG.write(Rsclr.AMReXSymb2Expr())

172

In [74]:
KLL = stVarRank2('KLL', sym = 'sym01')
for i in range(dim):
    for j in range(dim):
        KLL.expr[i][j] = AtildeLL.symb[i][j]/chi.symb+1/3*gamLL.symb[i][j]*(Khat.symb+2*theta.symb)

In [75]:
fileDIAG.write(KLL.AMReXSymb2Expr())

819

In [76]:
KUU = stVarRank2('KUU', sym = 'sym01')
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            for l in range(dim):
                KUU.expr[i][j] += gamUU.symb[i][k]*gamUU.symb[j][l]*KLL.symb[k][l]

In [77]:
fileDIAG.write(KUU.AMReXSymb2Expr())

2232

In [78]:
H.expr = Rsclr.symb + (Khat.symb + 2*theta.symb)**2

for i in range(dim):
    for j in range(dim):
        H.expr += -KLL.symb[i][j]*KUU.symb[i][j]

In [79]:
fileDIAG.write(H.AMReXSetDiag())

198

In [80]:
fileDIAG.write(H.AMReXSymb2State())

49

In [81]:
InterKLL = stVarRank2('InterKLL', sym = 'sym01')
for i in range(dim):
    for j in range(dim):
        InterKLL.expr[i][j] = KLL.symb[i][j]-gamLL.symb[i][j]*(Khat.symb + 2*theta.symb)

In [82]:
fileDIAG.write(InterKLL.AMReXSymb2Expr())

603

In [83]:
dInterKLLL = stVarRank3('dInterKLLL')
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            dInterKLLL.expr[i][j][k] += (1/chi.symb)*(-Ds(chi,str(k))*InterKLL.symb[i][j]+Dsymb(AtildeLL.symb[i][j],str(k))-2/3*Dsymb(gamtildeLL.symb[i][j],str(k))*(Khat.symb+2*theta.symb)-2/3*gamtildeLL.symb[i][j]*(Ds(Khat,str(k))+2*Ds(theta,str(k))))

In [84]:
fileDIAG.write(dInterKLLL.AMReXSymb2Expr())

5022

In [85]:
DInterKL = stVarRank1('DInterKL')
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            DInterKL.expr[i] += gamUU.symb[j][k]*dInterKLLL.symb[i][j][k]
            for l in range(dim):
                DInterKL.expr[i] += -gamUU.symb[j][k]*(GamULL.symb[l][i][k]*InterKLL.symb[l][j]+GamULL.symb[l][j][k]*InterKLL.symb[i][l])

In [86]:
fileDIAG.write(DInterKL.AMReXSymb2Expr())

5073

In [87]:
ML.expr = DInterKL.symb

In [88]:
fileDIAG.write(ML.AMReXSetDiag())

141

In [89]:
fileDIAG.write(ML.AMReXSymb2State())

156

In [90]:
MU = stVarRank1('MU')
for i in range(dim):
    for j in range(dim):
        MU.expr[i] += gamUU.symb[i][j]*ML.symb[j]

In [91]:
fileDIAG.write(MU.AMReXSymb2Expr())

201

In [92]:
Csqr = stVar('Csqr')

In [93]:
Csqr.expr = H.symb**2 + theta.symb**2
for i in range(dim):
    Csqr.expr += MU.symb[i]*ML.symb[i]    

In [94]:
fileDIAG.write(Csqr.AMReXSymb2Expr())

95

In [95]:
C.expr = sp.sqrt(Csqr.symb)

In [96]:
fileDIAG.write(C.AMReXSetDiag())

51

In [97]:
Ksclr.expr = Khat.symb + 2*theta.symb

In [98]:
fileDIAG.write(Ksclr.AMReXSetDiag())

54

In [99]:
fileDIAGVARS = open(path+"ET_Integration_Diagnostic_Variables.H","w+")
fileDIAGVARS.write(stfw.VarString(stVar.declDiag))
fileDIAGVARS.close()

In [100]:
fileDIAG.write(stfw.Closer())
fileDIAG.close()

# Load Initial Data file (must have same parameters as Initial Data RHS)

In [101]:
from sympy import Eijk, LeviCivita

In [102]:
stVar.declInit = []
u = stVar('u', initial = True)

In [103]:
fileLOADINIT = open(path+"ET_Integration_LoadInitData_K.H", "w+")

In [104]:
fileLOADINIT.write(stfw.LoadInit_Header())

744

In [105]:
fileLOADINIT.write(u.AMReXSymb2State())

60

In [106]:
x = stVarRank1('x')  #Defining symbols for the grid point positions.  They are already declared in the header.
r = stVar('r')  #Define symbol for radius from the center of our coordinate system.
r.expr = sp.sqrt(x.symb[0]**2+x.symb[1]**2+x.symb[2]**2)  #Defining how r is related to x0, x1, and x2.

#We place two black holes at postions along the x0 = x axis.
c1 = stVarRank1('c1')
c2 = stVarRank1('c2')
c1.expr = np.array([1.168642873,0,0])
c2.expr = np.array([-1.168642873,0,0])

#Bare masses
M1 = stVar('M1')
M2 = stVar('M2')

M1.expr = 0.453
M2.expr = 0.453

#Momenta
P1 = stVarRank1('P1')
P2 = stVarRank1('P2')

P1.expr = np.array([0,0.3331917498,0])
P2.expr = np.array([0,-0.3331917498,0])

S1 = stVarRank1('S1')
S2 = stVarRank1('S2')

S1.expr = [0,0,0]
S2.expr = [0,0,0]

#To simplify expressions below we can specify intermediate coordinates defined relative to the black hole centers.
xc1 = stVarRank1('xc1')
xc2 = stVarRank1('xc2')

for i in range(3):
    xc1.expr[i] = x.symb[i]-c1.symb[i]
    xc2.expr[i] = x.symb[i]-c2.symb[i]

rc1 = stVar('rc1')
rc1.expr = sp.Max(sp.sqrt(xc1.symb[0]**2 + xc1.symb[1]**2 + xc1.symb[2]**2), 0.01)

rc2 = stVar('rc2')
rc2.expr = sp.Max(sp.sqrt(xc2.symb[0]**2 + xc2.symb[1]**2 + xc2.symb[2]**2), 0.01)

n1 = stVarRank1('n1')
n2 = stVarRank1('n2')

for i in range(3):
    n1.expr[i] = xc1.symb[i]/rc1.symb
    n2.expr[i] = xc2.symb[i]/rc2.symb

In [107]:
fileLOADINIT.write(r.AMReXSymb2Expr())
fileLOADINIT.write(c1.AMReXSymb2Expr())
fileLOADINIT.write(c2.AMReXSymb2Expr())
fileLOADINIT.write(M1.AMReXSymb2Expr())
fileLOADINIT.write(M2.AMReXSymb2Expr())
fileLOADINIT.write(P1.AMReXSymb2Expr())
fileLOADINIT.write(P2.AMReXSymb2Expr())
fileLOADINIT.write(S1.AMReXSymb2Expr())
fileLOADINIT.write(S2.AMReXSymb2Expr())
fileLOADINIT.write(xc1.AMReXSymb2Expr())
fileLOADINIT.write(xc2.AMReXSymb2Expr())
fileLOADINIT.write(rc1.AMReXSymb2Expr())
fileLOADINIT.write(rc2.AMReXSymb2Expr())
fileLOADINIT.write(n1.AMReXSymb2Expr())
fileLOADINIT.write(n2.AMReXSymb2Expr())

108

In [108]:
Abar1UU = stVarRank2('Abar1UU')
Abar2UU = stVarRank2('Abar2UU')
for i in range(3):
    for j in range(3):
        Abar1UU.expr[i][j] += 3/(2*(rc1.symb)**2)*(P1.symb[i]*n1.symb[j]+P1.symb[j]*n1.symb[i])
        Abar2UU.expr[i][j] += 3/(2*(rc2.symb)**2)*(P2.symb[i]*n2.symb[j]+P2.symb[j]*n2.symb[i])
        for k in range(3):
            Abar1UU.expr[i][j] += -3/(2*(rc1.symb)**2)*(sp.eye(3)[i,j]-n1.symb[i]*n1.symb[j])*n1.symb[k]*P1.symb[k]
            Abar2UU.expr[i][j] += -3/(2*(rc2.symb)**2)*(sp.eye(3)[i,j]-n2.symb[i]*n2.symb[j])*n2.symb[k]*P2.symb[k]
            
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                Abar1UU.expr[i][j] +=3/(rc1.symb**3)*(n1.symb[i]*Eijk(j,k,l)*S1.symb[k]*n1.symb[l]+n1.symb[j]*Eijk(i,k,l)*S1.symb[k]*n1.symb[l])
                Abar2UU.expr[i][j] +=3/(rc2.symb**3)*(n2.symb[i]*Eijk(j,k,l)*S2.symb[k]*n2.symb[l]+n2.symb[j]*Eijk(i,k,l)*S2.symb[k]*n2.symb[l])

In [109]:
AbarUU = stVarRank2('AbarUU')
for i in range(3):
    for j in range(3):
        AbarUU.expr[i][j] += Abar1UU.symb[i][j]+Abar2UU.symb[i][j]

In [110]:
fileLOADINIT.write(Abar1UU.AMReXSymb2Expr())
fileLOADINIT.write(Abar2UU.AMReXSymb2Expr())
fileLOADINIT.write(AbarUU.AMReXSymb2Expr())

486

In [111]:
alphaFunc = stVar('alphaFunc')
alphaFunc.expr = 1/(M1.symb/(2*rc1.symb)+M2.symb/(2*rc2.symb))

In [112]:
fileLOADINIT.write(alphaFunc.AMReXSymb2Expr())

76

In [113]:
psiInit = stVar('psiInit')
psiInit.expr = 1 + u.symb + 1/alphaFunc.symb

In [114]:
fileLOADINIT.write(psiInit.AMReXSymb2Expr())

54

In [115]:
chiInit = stVar('chi')
chiInit.expr = psiInit.symb**(-4)

In [116]:
fileLOADINIT.write(chiInit.AMReXSetState())

63

In [117]:
gamtildeLLInit = stVarRank2('gamtildeLL', sym = 'sym01')
gamtildeLLInit.expr = [[1,0,0],[0,1,0],[0,0,1]]

In [118]:
fileLOADINIT.write(gamtildeLLInit.AMReXSetState())

312

In [119]:
KhatInit = stVar('Khat')
KhatInit.expr = 0

In [120]:
fileLOADINIT.write(KhatInit.AMReXSetState())

44

In [121]:
AtildeLLInit = stVarRank2('AtildeLL', sym = 'sym01')

AtildeLLInit.expr = psiInit.symb**(-6)*AbarUU.symb

In [122]:
fileLOADINIT.write(AtildeLLInit.AMReXSetState())

468

In [123]:
thetaInit = stVar('theta')
thetaInit.expr = 0

In [124]:
fileLOADINIT.write(thetaInit.AMReXSetState())

45

In [125]:
GamtildeUInit = stVarRank1('GamtildeU')
GamtildeUInit.expr = [0,0,0]

In [126]:
fileLOADINIT.write(GamtildeUInit.AMReXSetState())

150

In [127]:
alphaInit = stVar('alpha')
alphaInit.expr = psiInit.symb**(-2)

In [128]:
fileLOADINIT.write(alphaInit.AMReXSetState())

65

In [129]:
betaUInit = stVarRank1('betaU')
betaUInit.expr = [0,0,0]

In [130]:
fileLOADINIT.write(betaUInit.AMReXSetState())

138

In [131]:
fileLOADINIT.write(stfw.Closer())
fileLOADINIT.close()

# Setup file

In [132]:
fileSETUP = open(path+"ET_Integration_Setup_K.H", "w+")
fileSETUP.write(stfw.Write_Setup_File(stVar.declState, stVar.declInit, stVar.declDiag, nghostcells))
fileSETUP.close()

# Post timestep updating

In [133]:
filePOST = open(path+"ET_Integration_Post_Update_K.H", "w+")

In [134]:
filePOST.write(stfw.PostUpdate_Header())

708

In [135]:
filePOST.write(gamtildeLL.AMReXSymb2State())

666

In [136]:
detgamtilde = stVar('detgamtilde')
detgamtilde.expr = sp.simplify(sp.Matrix(gamtildeLL.symb).det())

In [137]:
filePOST.write(detgamtilde.AMReXSymb2Expr())

280

In [138]:
scalefactor = stVar('scalefactor')
scalefactor.expr = detgamtilde.symb**(-1.0/3.0)

In [139]:
filePOST.write(scalefactor.AMReXSymb2Expr())

80

In [140]:
Reset_gamtildeLL = stVarRank2('gamtildeLL',sym = 'sym01')
Reset_gamtildeLL.expr = gamtildeLL.symb*scalefactor.symb

In [141]:
filePOST.write(Reset_gamtildeLL.AMReXSetState())

450

In [142]:
filePOST.write(AtildeLL.AMReXSymb2State())

630

In [143]:
filePOST.write(gamtildeUU.AMReXSymb2Expr())

3026

In [144]:
TrAtilde = stVar('TrAtilde')
for i in range(3):
    for j in range(3):
        TrAtilde.expr += gamtildeUU.symb[i][j]*AtildeLL.symb[i][j]

In [145]:
filePOST.write(TrAtilde.AMReXSymb2Expr())

265

In [146]:
Reset_AtildeLL = stVarRank2('AtildeLL', sym = 'sym01')
Reset_AtildeLL.expr = AtildeLL.symb -1/3*TrAtilde.symb*gamtildeLL.symb

In [147]:
filePOST.write(Reset_AtildeLL.AMReXSetState())

618

In [148]:
filePOST.write(stfw.Closer())
filePOST.close()

# INIT file (empty for this case)

In [149]:
fileINIT = open(path+"ET_Integration_Init_K.H", "w+")

In [150]:
fileINIT.write(stfw.InitFromScratch_Header())

612

In [151]:
fileINIT.write(stfw.Closer())
fileINIT.close()

# AMR taging file

In [152]:
fileAMRTAG = open(path+"ET_Integration_AMR_Cell_Tag_K.H", "w+")

In [153]:
fileAMRTAG.write(stfw.AMRtagging_Header())

744

In [154]:
fileAMRTAG.write(alpha.AMReXSymb2State())

61

In [155]:
TagVar = stVar('TagVar')
TagVar.expr = str(sp.Abs(alpha.symb))

In [156]:
fileAMRTAG.write(TagCondition(TagVar, '<'))

51

In [157]:
fileAMRTAG.write(stfw.Closer())
fileAMRTAG.close()

# Gridding, Parameters, and Inputs

In [158]:
import STvARInputs as sti

In [159]:
pathInputs = "../Exec/Z4c/"
INPUTS = open(pathInputs + "inputs", "w+")

In [160]:
#STOPPING CRITERIA
MAX_STEP = 200
MAX_TIME = 100.0

#Equation type (default hyperbolic) and Courant Factor.
CFL = 0.25

#Geometry
NUM_COARSE_CELLS = [64, 64, 64]
LOWER_BOUNDS = [-50.0, -50.0, -50.0]
UPPER_BOUNDS = [50.0, 50.0, 50.0]
PERIODICITY = [0, 0, 0]

#AMR Refinement Parameters
NUM_LEVELS = 6
MAX_NUM_GRID_CELLS = 64
REF_RATIO = 2
NUM_BUFFER_CELLS = 1
BLOCKING_FACTOR = 8
REGRID_PERIOD = 1

#Coarse/Fine boundary interpolation procedure
INTERPOLATION_TYPE = 6

TAGGING_THRESHOLDS = [0.7, 0.6, 0.5, 0.4, 0.3]
ERROR_COMP = 0

#Problem specific Boundary Conditions
BC_UPPER = [2, 2, 2]
BC_LOWER = [2, 2, 2]

#How often to plot states, plot diagnostics, and write checkpoint files
PLOT_INT = 1
DIAG_INT = 1
CHECKPOINT_INT = 10

#Restart file names
RESTART_FILE = "initial_data_chk"
IS_INITIAL_DATA = 1

#Integration type.
RUNGE_KUTTA_TYPE = 4

In [161]:
INPUTS.write(sti.StoppingCriteria(MAX_STEP, MAX_TIME))
INPUTS.write(sti.Geometry(NUM_COARSE_CELLS, LOWER_BOUNDS, UPPER_BOUNDS, PERIODICITY))
INPUTS.write(sti.AMRVerbosity())
INPUTS.write(sti.RefinementPar(NUM_LEVELS, MAX_NUM_GRID_CELLS, REF_RATIO, NUM_BUFFER_CELLS, BLOCKING_FACTOR, REGRID_PERIOD))
INPUTS.write(sti.Interpolation(INTERPOLATION_TYPE))
INPUTS.write(sti.BoundaryConditions(BC_LOWER, BC_UPPER))
INPUTS.write(sti.TaggingThresholds(TAGGING_THRESHOLDS, ERROR_COMP))
INPUTS.write(sti.ProblemType(CFL))
INPUTS.write(sti.PlottingCriteria(PLOT_INT, DIAG_INT, CHECKPOINT_INT))
INPUTS.write(sti.RestartFromCheckPoint(RESTART_FILE, IS_INITIAL_DATA))
INPUTS.write(sti.IntegrationType(RUNGE_KUTTA_TYPE))

711

In [162]:
INPUTS.close()

# To be added

Less dependence on NRPy+ functions